In [ ]:
# If you're on Colab, UNCOMMENT the next line then run this cell once:
# !pip -q install pandas numpy scikit-learn plotly tqdm textblob arabic-reshaper python-bidi

import pandas as pd, numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
from tqdm import tqdm
import re, math, random, json

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)


In [ ]:
CSV_PATH = "tuniind.csv"  # change if your file has another name

# In Colab you can run files.upload() to pick a file:
try:
    from google.colab import files
    print("Use files.upload() below if the file isn't in the workspace.")
    # files.upload()  # <- UNCOMMENT to upload from your device
except Exception:
    pass

print("Looking for:", CSV_PATH)
df_raw = pd.read_csv(CSV_PATH, encoding="utf-8", sep=",", engine="python", on_bad_lines="skip")
print("Loaded shape:", df_raw.shape)
df_raw.head(3)


Use files.upload() below if the file isn't in the workspace.
Looking for: tuniind.csv
Loaded shape: (597, 20)


,Dénomination,Raison Sociale,Responsable,Secteur,Activités,Produits,Adresse usine,District,Gouvernorat,Délégation,Téléphone siège/usine,Fax siège/usine,E-mail,URL,Régime,Pays du Participant \n Etranger,Entrée en production,Capital en DT,Emploi,Banque_Tunisienne_Associée
0,7 M,STE 7 M,Sami Ben Said,Industries agro-alimentaires,"Laiteries, fromageries et yaourterie.","Fromages à pâte molle, pressée, persillée à pâ...",DISTRICT INDUSTRIEL D'ENFIDHA - 4030 - ENFIDHA,"Troisième district : Kairouan, Kasserine, Mahd...",Sousse,Enfidha,(216) - 20 578 142 / 94 493 691,NaN,sami.abdennour@societe7m.com,NaN,Totalement exportatrice,Libye - Allemagne,20/05/2019,1 500 000,18.0,Banque Zitouna
1,ABATTOIR AVICOLE ESSOUASSI,STE ABATTOIR AVICOLE ESSOUASSI,Fares Ghannouchi,Industries agro-alimentaires,Viandes de volailles et de lapins.,Viandes de volailles et de lapins.,Z.I. AFI - 5140 - SOUASSI,"Troisième district : Kairouan, Kasserine, Mahd...",Mahdia,Essouassi,(216) - 29 327 113 / 24 327 027,(216) - 73 611 147,commercial.saas2007@gmail.com,NaN,Non totalement exportatrice,NaN,01/09/2009,1 020 000,140.0,Attijari Bank
2,ABCO,AGRI BUSINESS COMPANY,Ezzeddine Bellagha,Industries agro-alimentaires,"Transformation, conservation, congélation et s...",Conserves de poissons - Farines de poisson.,PORT DE PECHE - 8016 - SIDI DAOUD,"Deuxième district : Ariana, Ben Arous, Manouba...",Nabeul,El haouaria,(216) - 71 454 833 / 72 294 515,(216) - 71 454 832 / 72 294 984,abco@abco.tn,http://www.sididaoud.com,Non totalement exportatrice,NaN,07/12/2000,7 500 000,400.0,Société tunisienne de banque


In [ ]:
def normalize_columns(df):
    df2 = df.copy()
    df2.columns = (
        df2.columns.str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^\w_]", "", regex=True)
        .str.lower()
    )
    return df2

df = normalize_columns(df_raw)

possible_name_cols   = [c for c in df.columns if c in ["dénomination","denomination","nom","raison_sociale","denomination_sociale","denomination"]]
possible_city_cols   = [c for c in df.columns if c in ["gouvernorat","governorate","gouvernorat_du_siege","adresse","ville","region","wilaya"]]
possible_sector_cols = [c for c in df.columns if c in ["activites","activite","activité","secteur","activites_principales","secteur_dactivite","categorie"]]

df["name"]         = df[possible_name_cols[0]] if possible_name_cols else df.iloc[:,0].astype(str)
df["governorate"]  = df[possible_city_cols[0]] if possible_city_cols else "Unknown"
df["sector_text"]  = df[possible_sector_cols[0]] if possible_sector_cols else ""

# drop dupes
df["name_key"] = df["name"].str.lower().str.replace(r"[^a-z0-9]+","", regex=True)
df = df.drop_duplicates(subset=["name_key","governorate"]).drop(columns=["name_key"], errors="ignore")

gov_map = {
 "tunis":"Tunis","sfax":"Sfax","sousse":"Sousse","kairouan":"Kairouan","bizerte":"Bizerte",
 "gabes":"Gabès","ariana":"Ariana","ben arous":"Ben Arous","nabeul":"Nabeul","manouba":"Manouba",
 "kasserine":"Kasserine","kebili":"Kébili","medenine":"Médenine","tataouine":"Tataouine",
 "gafsa":"Gafsa","siliana":"Siliana","jendouba":"Jendouba","beja":"Béja","zaghouan":"Zaghouan",
 "monastir":"Monastir","mahdia":"Mahdia","tozeur":"Tozeur","sidi bouzid":"Sidi Bouzid","la marsa":"La Marsa"
}
df["governorate"] = df["governorate"].astype(str).str.strip().apply(lambda x: gov_map.get(x.lower(), x))
print("Clean shape:", df.shape)
df[["name","governorate","sector_text"]].head(5)


Clean shape: (595, 23)


,name,governorate,sector_text
0,7 M,Sousse,Industries agro-alimentaires
1,ABATTOIR AVICOLE ESSOUASSI,Mahdia,Industries agro-alimentaires
2,ABCO,Nabeul,Industries agro-alimentaires
3,ABDELAZIZ MEDINI,Béja,Industries agro-alimentaires
4,ABID DE PRODUCTION D'HUILE EXTRA,Sfax,Industries agro-alimentaires


In [ ]:
import re
import numpy as np

text = df["sector_text"].fillna("").astype(str).str.lower()

def contains_any(s, keywords):
    return int(any(k in s for k in keywords))

# --- Digital adoption proxy (0 or 1) ---
digital_keywords = ["tpe","pos","d17","ecommerce","e-commerce","carte","qr",
                    "paiement","online","virement","wallet"]
df["digital_adoption"] = text.apply(lambda s: contains_any(s, digital_keywords)).astype(float)

# --- Cash usage proxy ---
cash_keywords = ["cash seulement","espèces uniquement","cash only"]
df["cash_usage_ratio"] = text.apply(
    lambda s: 1.0 if contains_any(s, cash_keywords) else 0.3 + 0.4*np.random.rand()
)

# --- Innovation proxy ---
innov_keywords = ["innovation","nouveau","digital","saas","cloud","fintech","plateforme"]
df["innovation_seeking_score"] = text.apply(
    lambda s: 0.7 if contains_any(s, innov_keywords) else 0.4 + 0.2*np.random.rand()
)

# --- Loyalty score (random between 0.3 and 0.9) ---
df["loyalty_score"] = np.random.uniform(0.3, 0.9, size=len(df))

# --- Clean and coerce capital_en_dt ---
def parse_money(x):
    if pd.isna(x):
        return np.nan
    s = str(x).lower().strip()
    s = s.replace("dt","").replace("tnd","").replace("dinars","").replace("dinar","")
    s = s.replace("million","m").replace("millions","m").replace("mdt","m").replace("md","m")
    s = s.replace("kdt","k").replace(",", ".").replace(" ", "")
    mult = 1.0
    if s.endswith("m"): mult, s = 1_000_000, s[:-1]
    elif s.endswith("k"): mult, s = 1_000, s[:-1]
    s = re.sub(r"[^0-9.\-eE]", "", s)
    try:
        return float(s) * mult
    except:
        return np.nan

if "capital_en_dt" in df.columns:
    df["capital_en_dt"] = df["capital_en_dt"].apply(parse_money)

if "capital_en_dt" not in df.columns:
    df["capital_en_dt"] = np.random.lognormal(mean=11, sigma=0.8, size=len(df))
else:
    med_cap = pd.to_numeric(df["capital_en_dt"], errors="coerce").median()
    df["capital_en_dt"] = pd.to_numeric(df["capital_en_dt"], errors="coerce").fillna(med_cap)
df["capital_en_dt"] = df["capital_en_dt"].clip(1e3, 1e9)

# --- Employment column ---
if "emploi" not in df.columns:
    df["emploi"] = np.random.lognormal(mean=3.5, sigma=0.6, size=len(df)).astype(int)
else:
    df["emploi"] = pd.to_numeric(df["emploi"], errors="coerce")
    med_emp = df["emploi"].median()
    df["emploi"] = df["emploi"].fillna(med_emp).astype(int)

# --- Credit worthiness ---
df["credit_worthiness"] = (
    0.4*np.log1p(df["capital_en_dt"]) +
    0.3*(1 - df["cash_usage_ratio"]) +
    0.3*df["loyalty_score"]
)

# --- Final features ---
feature_cols = ["capital_en_dt","emploi","credit_worthiness","loyalty_score","digital_adoption"]
df_feats = df[feature_cols].copy()
df_feats.head(3)


,capital_en_dt,emploi,credit_worthiness,loyalty_score,digital_adoption
0,1500000.0,18,5.955569,0.340410,0.0
1,1020000.0,140,5.824831,0.649302,0.0
2,7500000.0,400,6.606585,0.507530,0.0


In [ ]:
df


,dénomination,raison_sociale,responsable,secteur,activités,produits,adresse_usine,district,gouvernorat,délégation,...,emploi,banque_tunisienne_associée,name,governorate,sector_text,digital_adoption,cash_usage_ratio,innovation_seeking_score,loyalty_score,credit_worthiness
0,7 M,STE 7 M,Sami Ben Said,Industries agro-alimentaires,"Laiteries, fromageries et yaourterie.","Fromages à pâte molle, pressée, persillée à pâ...",DISTRICT INDUSTRIEL D'ENFIDHA - 4030 - ENFIDHA,"Troisième district : Kairouan, Kasserine, Mahd...",Sousse,Enfidha,...,18,Banque Zitouna,7 M,Sousse,Industries agro-alimentaires,0.0,0.449816,0.494792,0.340410,5.955569
1,ABATTOIR AVICOLE ESSOUASSI,STE ABATTOIR AVICOLE ESSOUASSI,Fares Ghannouchi,Industries agro-alimentaires,Viandes de volailles et de lapins.,Viandes de volailles et de lapins.,Z.I. AFI - 5140 - SOUASSI,"Troisième district : Kairouan, Kasserine, Mahd...",Mahdia,Essouassi,...,140,Attijari Bank,ABATTOIR AVICOLE ESSOUASSI,Mahdia,Industries agro-alimentaires,0.0,0.680286,0.533512,0.649302,5.824831
2,ABCO,AGRI BUSINESS COMPANY,Ezzeddine Bellagha,Industries agro-alimentaires,"Transformation, conservation, congélation et s...",Conserves de poissons - Farines de poisson.,PORT DE PECHE - 8016 - SIDI DAOUD,"Deuxième district : Ariana, Ben Arous, Manouba...",Nabeul,El haouaria,...,400,Société tunisienne de banque,ABCO,Nabeul,Industries agro-alimentaires,0.0,0.592798,0.434464,0.507530,6.606585
3,ABDELAZIZ MEDINI,ABDELAZIZ MEDINI,Walid Medini,Industries agro-alimentaires,Aliments pour animaux de ferme.,Aliments complets pour les animaux de ferme.,AMDOUN - 9030 - AMDOUN,"Premier district : Bizerte, Béja, Jendouba et ...",Beja,Amdoun,...,11,Banque nationale agricole,ABDELAZIZ MEDINI,Béja,Industries agro-alimentaires,0.0,0.539463,0.438458,0.672549,5.740246
4,ABID DE PRODUCTION D'HUILE EXTRA,STE ABID DE PRODUCTION D'HUILE EXTRA,Faouzi Abid,Industries agro-alimentaires,Huiles d'olives et conditionnement.,Huiles d'olive.,RTE GREMDA KM 9 - 3000 - SFAX,"Quatrième district : Gafsa, Sfax, Sidi Bouzid ...",Sfax,Sfax sud,...,10,Attijari Bank,ABID DE PRODUCTION D'HUILE EXTRA,Sfax,Industries agro-alimentaires,0.0,0.362407,0.408174,0.327445,4.967613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,CAVEO AUTOMATIVE TUNISIA,CAVEO AUTOMATIVE TUNISIA SA,Nabhen Bouchaala,Industries mécaniques et métallurgiques,Equipements automobiles.,Suspensions.,"Z.I. BORJ CEDRIA BP 912, RUE NELSON MANDELLA -...","Deuxième district : Ariana, Ben Arous, Manouba...",Ben Arous,Hammam chott,...,389,Amen Bank,CAVEO AUTOMATIVE TUNISIA,Ben Arous,Industries mécaniques et métallurgiques,0.0,0.355531,0.587446,0.323056,7.419395
593,CEFER,CEFER,Zied Abdennadher,Industries mécaniques et métallurgiques,Ouvrages divers en métaux.,Autres ouvrages divers en métaux.,RTE DE TUNIS KM 22 Z.I. - 3043 - EL GHRABA,"Quatrième district : Gafsa, Sfax, Sidi Bouzid ...",Sfax,El hencha,...,26,Arab Tunisian Bank,CEFER,Sfax,Industries mécaniques et métallurgiques,0.0,0.556350,0.537577,0.841057,5.707287
594,CEM (SAKIET EZZIT),CONSTRUCTION ET EMBOUTISSAGE METALLIQUES,Moncef Hadj Kacem,Industries mécaniques et métallurgiques,Ouvrages divers en métaux - Carrosseries et re...,Carrosseries pour véhicules automobiles - Pare...,RTE DE GREMDA KM 9 - 3000 - SFAX,"Quatrième district : Gafsa, Sfax, Sidi Bouzid ...",Sfax,Sfax sud,...,40,Arab Tunisian Bank,CEM (SAKIET EZZIT),Sfax,Industries mécaniques et métallurgiques,0.0,0.372752,0.413567,0.576886,5.887445
595,CENTRAL,STE CENTRAL,Khalil Said,Industries mécaniques et métallurgiques,Menuiseries et fermetures métalliques.,Menuiserie aluminium.,"RTE DE TUNIS, Z.I. - 4022 - AKOUDA","Troisième district : Kairouan, Kasserine, Mahd...",Sousse,Akouda,...,40,Banque de Tunisie,CENTRAL,Sousse,Industries mécaniques et métallurgiques,0.0,0.438267,0.460193,0.682321,5.255647


In [ ]:
df["credit_worthiness"] = (
    0.4*np.log1p(df["capital_en_dt"]) +
    0.3*(1 - df["cash_usage_ratio"]) +
    0.3*df["loyalty_score"]
)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df_feats)

best_k, best_score, best_model = None, -1, None
for k in range(2, 7):
    km = KMeans(n_clusters=k, random_state=RANDOM_SEED, n_init="auto")
    labels = km.fit_predict(X)
    score = silhouette_score(X, labels)
    if score > best_score:
        best_k, best_score, best_model = k, score, km

df["cluster"] = best_model.predict(X)
print(f"Chosen clusters: K={best_k}, silhouette={best_score:.3f}")
df.groupby("cluster")[feature_cols].mean().round(2)


Chosen clusters: K=2, silhouette=0.664


,capital_en_dt,emploi,credit_worthiness,loyalty_score,digital_adoption
cluster,,,,,
0,57678880.37,564.26,7.11,0.55,0.0
1,2373645.16,48.30,5.50,0.60,0.0


In [ ]:
fig = px.scatter_3d(
    df.sample(min(400, len(df))),
    x="capital_en_dt", y="emploi", z="credit_worthiness",
    color="cluster",
    hover_data=["name","governorate","sector_text"],
    opacity=0.7
)
fig.update_traces(marker=dict(size=4))
fig.show()


In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Agent:
    id: int; name: str; governorate: str; cluster: int
    cash_usage_ratio: float; digital_adoption: float
    loyalty_score: float; credit_worthiness: float
    capital_en_dt: float; emploi: int; innovation_seeking_score: float
    conversion_probability: float = 0.0
    decision_trace: List[str] = field(default_factory=list)

agents = []
for i, row in df.reset_index(drop=True).iterrows():
    agents.append(Agent(
        id=int(i), name=str(row["name"])[:120], governorate=row["governorate"], cluster=int(row["cluster"]),
        cash_usage_ratio=float(row["cash_usage_ratio"]), digital_adoption=float(row["digital_adoption"]),
        loyalty_score=float(row["loyalty_score"]), credit_worthiness=float(row["credit_worthiness"]),
        capital_en_dt=float(row["capital_en_dt"]), emploi=int(row["emploi"]),
        innovation_seeking_score=float(row["innovation_seeking_score"]),
    ))
len(agents)


595

In [ ]:
def score_agent(agent, offer_strength=0.0):
    p, tr = 0.0, []
    if agent.cash_usage_ratio > 0.7:   p -= 0.5; tr.append("High cash usage → lower conversion (-0.5)")
    elif agent.cash_usage_ratio > 0.5: p -= 0.2; tr.append("Moderate cash usage (-0.2)")
    else:                               p += 0.05; tr.append("Low cash usage (+0.05)")

    if agent.digital_adoption > 0.5:    p += 0.4; tr.append("Digital adoption high (+0.4)")
    else:                               p += 0.05; tr.append("Digital adoption low (+0.05)")

    p += 0.001 * (agent.credit_worthiness)
    p += 0.2 * (agent.loyalty_score - 0.5)
    p += offer_strength
    p = float(max(0.0, min(1.0, 0.5 + p)))  # clamp
    return p, tr

for a in agents:
    p, tr = score_agent(a, offer_strength=0.0)
    a.conversion_probability, a.decision_trace = p, tr

pd.DataFrame([{"id":a.id,"name":a.name,"p_conv":a.conversion_probability,"cluster":a.cluster} for a in agents]).head(10)


,id,name,p_conv,cluster
0,0,7 M,0.574038,1
1,1,ABATTOIR AVICOLE ESSOUASSI,0.385685,1
2,2,ABCO,0.358113,1
3,3,ABDELAZIZ MEDINI,0.390250,1
4,4,ABID DE PRODUCTION D'HUILE EXTRA,0.570457,1
5,5,AC (SIDI BOUZID),0.670423,1
6,6,ACN,0.683655,1
7,7,ADAM EXPORT,0.431885,1
8,8,AFRICA HUILE,0.404837,1
9,9,AFROST COMPAGNY,0.331610,1


In [ ]:
SCENARIOS = {
    "Baseline": {"offer_strength": 0.0,  "digital_shift": 0.00, "risk_penalty": 0.00},
    "Economic Recovery": {"offer_strength": 0.05, "digital_shift": 0.05, "risk_penalty": -0.02},
    "Export Boom": {"offer_strength": 0.08, "digital_shift": 0.03, "risk_penalty": -0.01},
    "Digital Transformation": {"offer_strength": 0.06, "digital_shift": 0.10, "risk_penalty": -0.03},
    "Regional Instability": {"offer_strength": -0.03,"digital_shift": -0.02,"risk_penalty": 0.08},
    "Tourism Recovery": {"offer_strength": 0.04, "digital_shift": 0.01, "risk_penalty": -0.01},
    "Currency Devaluation": {"offer_strength": -0.06,"digital_shift": 0.02, "risk_penalty": 0.10},
    "Energy Crisis": {"offer_strength": -0.10,"digital_shift": -0.03,"risk_penalty": 0.15},
    "Political Uncertainty": {"offer_strength": -0.02,"digital_shift": -0.01,"risk_penalty": 0.12},
}
SCENARIOS


{'Baseline': {'offer_strength': 0.0,
  'digital_shift': 0.0,
  'risk_penalty': 0.0},
 'Economic Recovery': {'offer_strength': 0.05,
  'digital_shift': 0.05,
  'risk_penalty': -0.02},
 'Export Boom': {'offer_strength': 0.08,
  'digital_shift': 0.03,
  'risk_penalty': -0.01},
 'Digital Transformation': {'offer_strength': 0.06,
  'digital_shift': 0.1,
  'risk_penalty': -0.03},
 'Regional Instability': {'offer_strength': -0.03,
  'digital_shift': -0.02,
  'risk_penalty': 0.08},
 'Tourism Recovery': {'offer_strength': 0.04,
  'digital_shift': 0.01,
  'risk_penalty': -0.01},
 'Currency Devaluation': {'offer_strength': -0.06,
  'digital_shift': 0.02,
  'risk_penalty': 0.1},
 'Energy Crisis': {'offer_strength': -0.1,
  'digital_shift': -0.03,
  'risk_penalty': 0.15},
 'Political Uncertainty': {'offer_strength': -0.02,
  'digital_shift': -0.01,
  'risk_penalty': 0.12}}

In [ ]:
def run_scenario(agents, scenario_key, mc_runs=200, months=12):
    params = SCENARIOS[scenario_key]
    results = []
    for _ in tqdm(range(mc_runs), leave=False):
        total_conversions = 0
        for a in agents:
            dig  = max(0.0, min(1.0, a.digital_adoption + params["digital_shift"]))
            cash = max(0.0, min(1.0, a.cash_usage_ratio + params["risk_penalty"]*0.5))
            tmp = Agent(**a.__dict__); tmp.digital_adoption, tmp.cash_usage_ratio = dig, cash
            p, _ = score_agent(tmp, offer_strength=params["offer_strength"])
            convs = np.random.binomial(months, p*0.1)  # monthly attempts
            total_conversions += convs
        results.append(total_conversions)
    return np.array(results)

scenario_keys = list(SCENARIOS.keys())
scores = {k: run_scenario(agents, k, mc_runs=200, months=12) for k in scenario_keys}

summary = pd.DataFrame({
    "scenario": scenario_keys,
    "avg_conversions": [float(np.mean(scores[k])) for k in scenario_keys],
    "std": [float(np.std(scores[k])) for k in scenario_keys],
}).sort_values("avg_conversions", ascending=False)
summary


,scenario,avg_conversions,std
2,Export Boom,412.710,21.023699
3,Digital Transformation,403.065,20.265754
1,Economic Recovery,395.005,19.809214
5,Tourism Recovery,385.190,18.834381
0,Baseline,354.195,16.254137
4,Regional Instability,295.405,18.065464
8,Political Uncertainty,277.860,17.079824
6,Currency Devaluation,262.390,16.324457
7,Energy Crisis,213.150,13.549815


In [ ]:
fig = px.bar(summary, x="scenario", y="avg_conversions", error_y="std",
             title="Expected Conversions by Scenario (MC mean ± std)")
fig.update_layout(xaxis_title="Scenario", yaxis_title="Expected conversions (relative units)")
fig.show()


In [ ]:
agent_id = 12  # change ID to inspect
a = agents[agent_id]
print(a.name, "| Gov:", a.governorate, "| Cluster:", a.cluster, "| p:", round(a.conversion_probability,3))
print("\nDecision trace:")
for step in a.decision_trace:
    print("-", step)


AGRIMED | Gov: Sfax | Cluster: 1 | p: 0.32

Decision trace:
- Moderate cash usage (-0.2)
- Digital adoption low (+0.05)
